In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from timeit import default_timer as timer
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

Xtr_path = 'Xtr.csv'
Ytr_path = 'ytr.csv'
Xts_path = 'Xts.csv'

Xtr = np.loadtxt(Xtr_path, delimiter=",")
Ytr = np.loadtxt(Ytr_path, delimiter=",")
Xts = np.loadtxt(Xts_path, delimiter=",")

In [2]:
Xtr,Xts,Ytr,Yts = train_test_split(Xtr,Ytr,test_size=0.2,random_state=0,stratify=Ytr)

In [3]:
Xtr.shape


(8000, 16)

In [4]:
Ytr.shape


(8000,)

In [5]:
Xts.shape

(2000, 16)

In [6]:
Yts.shape

(2000,)

In [7]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

batch_size = 128

# TODO Convert the numpy arrays to tensors
Xtr_torch = torch.Tensor(Xtr)
ytr_torch = torch.Tensor(Ytr)
Xts_torch = torch.Tensor(Xts)
yts_torch = torch.Tensor(Yts)

# TODO Create a training/test dataset from the tensors
# train_ds = ...
# test_ds = ...
train_ds = TensorDataset(Xtr_torch, ytr_torch)
# Create a testing Dataset from the tensors
test_ds = TensorDataset(Xts_torch, yts_torch)


# TODO Create a training/test data loader from datasets
# train_loader = ...
# test_loader = ...
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True) 
test_loader = DataLoader(test_ds, batch_size=batch_size) 

In [8]:
d_in = Xtr.shape[1]
d_out = len(np.unique(Ytr))
nh=100
class DumbNet(nn.Module):
    def __init__(self):
        super(DumbNet, self).__init__()
        
        self.classifier=nn.Sequential(
            nn.Linear(d_in,165),
            nn.BatchNorm1d(165),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(165,190),
            nn.BatchNorm1d(190),
          
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(190,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
          
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
           
            nn.GELU(),
            nn.Linear(64,d_out), 
            nn.LogSoftmax(dim=1)
            
        
        
      
    )
        
      
       
       
        
      
    def forward(self,x):
        x=x.view(x.size(0),-1)
        x=self.classifier(x)
      
        return x

model = DumbNet()     

In [9]:
print(model)

DumbNet(
  (classifier): Sequential(
    (0): Linear(in_features=16, out_features=165, bias=True)
    (1): BatchNorm1d(165, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=165, out_features=190, bias=True)
    (5): BatchNorm1d(190, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): GELU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=190, out_features=128, bias=True)
    (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.01)
    (11): Linear(in_features=128, out_features=64, bias=True)
    (12): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): GELU()
    (14): Linear(in_features=64, out_features=26, bias=True)
    (15): LogSoftmax(dim=1)
  )
)


In [10]:
with torch.no_grad():
    ytr_net = model(torch.Tensor(Xtr)).detach().numpy().argmax(axis=1)

acc = np.mean(ytr_net==Ytr)
print('training accuracy: ',acc)

training accuracy:  0.038375


In [11]:
import torch.optim as optim

# TODO
# opt = ...
# criterion = ...

opt = optim.Adam(model.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()

In [12]:
a_tr_loss = []
a_tr_accuracy = []
a_ts_loss = []
a_ts_accuracy = []


num_epoch = 400


for epoch in range(num_epoch):
    model.train() # put model in training mode
    correct = 0 # initialize error counter
    total = 0 # initialize total counter
    # iterate over training set
    for train_iter, data in enumerate(train_loader):
        x_batch,y_batch = data
        y_batch = y_batch.type(torch.long)
        out = model(x_batch)
        # Compute Loss
        loss = criterion(out,y_batch)
        a_tr_loss.append(loss.item())
        # Compute gradients using back propagation
        opt.zero_grad()
        loss.backward()
        # Take an optimization 'step'
        opt.step()
        # Do hard classification: index of largest score
        _, predicted = torch.max(out.data, 1)
        # Compute number of decision errors
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
        
    a_tr_accuracy.append( 100*correct/total )
    
    model.eval() # put model in evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            labels = labels.type(torch.long)
            outputs = model(images)
            loss = criterion(outputs,labels)
            a_ts_loss.append(loss.item())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    a_ts_accuracy.append( 100*correct/total )
    # Print details every print_mod epoch
    print('Epoch: {0:2d}   Train Loss: {1:.3f}   '.format(epoch+1, a_tr_loss[epoch])
          +'Train Acc: {0:.2f}    Test Loss: {1:.3f}   '.format(a_tr_accuracy[epoch], a_ts_loss[epoch])
          +'Test Acc: {0:.2f}'.format(a_ts_accuracy[epoch]))
   

Epoch:  1   Train Loss: 3.306   Train Acc: 21.09    Test Loss: 2.104   Test Acc: 45.60
Epoch:  2   Train Loss: 3.362   Train Acc: 40.15    Test Loss: 2.361   Test Acc: 58.90
Epoch:  3   Train Loss: 3.288   Train Acc: 49.54    Test Loss: 2.307   Test Acc: 62.85
Epoch:  4   Train Loss: 3.302   Train Acc: 54.38    Test Loss: 2.168   Test Acc: 67.50
Epoch:  5   Train Loss: 3.313   Train Acc: 57.15    Test Loss: 2.131   Test Acc: 68.35
Epoch:  6   Train Loss: 3.243   Train Acc: 59.77    Test Loss: 2.142   Test Acc: 69.20
Epoch:  7   Train Loss: 3.205   Train Acc: 62.26    Test Loss: 2.068   Test Acc: 71.30
Epoch:  8   Train Loss: 3.168   Train Acc: 63.31    Test Loss: 2.190   Test Acc: 70.45
Epoch:  9   Train Loss: 3.214   Train Acc: 64.26    Test Loss: 2.242   Test Acc: 72.75
Epoch: 10   Train Loss: 3.207   Train Acc: 65.39    Test Loss: 2.045   Test Acc: 74.10
Epoch: 11   Train Loss: 3.147   Train Acc: 66.20    Test Loss: 2.167   Test Acc: 75.60
Epoch: 12   Train Loss: 3.152   Train Acc: 

In [13]:
with torch.no_grad():
    ytr_net = model(torch.Tensor(Xtr)).detach().numpy().argmax(axis=1)

acc = np.mean(ytr_net==Ytr)
print('training accuracy: ',acc)

training accuracy:  0.9725


In [14]:
# save the model using the extension .pth
save_path = 'model.pth'

# To save a PyTorch model, we first pass an input through the model, 
# and then save the "trace". 
# For this purpose, we can use any input. 
# We will create a random input with the proper dimension.
x = torch.randn(d_in) # random input
x = x[None,:] # add singleton batch index
with torch.no_grad():
    traced_cell = torch.jit.trace(model, (x))

# Now we save the trace
torch.jit.save(traced_cell, save_path)


In [15]:
!python {"validation.py " + save_path}

training accuracy =  0.9569
test label predictions saved in yts_hat.csv
